In [2]:
# Importing necessary modules
import subprocess
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException  # Import the TimeoutException for handling timeouts
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
import pandas as pd
import time

# Hyperparameters
webpage_load_time = 9  # Time to wait for webpage to load, in seconds
click_load_time = 5  # Time to wait after clicking a button, in seconds
username = ""  # Your username here
password = ""  # Your password here

# Toggles
testing_mode = False  # Set to False for live mode
skip_login = False  # Set to True to skip login
use_debugging_mode = True  # Set this to True to use Chrome in debugging mode, False to use regular WebDriver
fuzzy_matching_pct = 70

# Function for robustly waiting for an element to appear
def robust_wait_for_element(driver, by, value, timeout=10):
    end_time = time.time() + timeout
    while time.time() < end_time:
        try:
            element = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((by, value)))
            return element
        except Exception as e:
            print(f"Exception encountered: {e}. Retrying...")
            time.sleep(1)
    print(f"Element not found: {value}")
    return None

# Function to verify bet slip with fuzzy name matching
def verify_bet_slip(driver, expected_fighters):
    bet_slip_source = driver.page_source
    bet_slip_soup = BeautifulSoup(bet_slip_source, 'html.parser')
    bet_slip_names = bet_slip_soup.find_all("div", class_="pick-name")
    bet_slip_names = [name.text.strip() for name in bet_slip_names]
    
    for expected in expected_fighters:
        best_match = max(bet_slip_names, key=lambda x: fuzz.partial_ratio(x, expected), default=None)
        if not best_match or fuzz.partial_ratio(best_match, expected) < fuzzy_matching_pct:
            return False, expected
    return True, None

def is_bet_successful(driver, timeout=15):  # Increase timeout to 15 seconds
    success_message_xpath = "//figure[contains(@class, 'betslip-alert success')]//div[text()='Bets successfully placed']"
    return robust_wait_for_element(driver, By.XPATH, success_message_xpath, timeout=timeout) is not None

# Functions for checking login and logging in
def is_already_logged_in(driver):
    login_check_xpath = "/html/body/bx-site/ng-component/bx-header-ch/div/bx-header-user-menu-ch/div/div[1]/div/wt-lazy/bx-account-menu-navigation/div/section/div/div/div/div[1]/div[2]/button"
    return robust_wait_for_element(driver, By.XPATH, login_check_xpath, timeout=5) is not None

def login_to_site(driver, username, password):
    login_url = "https://www.bovada.lv/sports/ufc-mma?overlay=login"
    driver.get(login_url)
    time.sleep(5)
    username_field = robust_wait_for_element(driver, By.XPATH, "/html/body/bx-site/bx-overlay-ch/bx-overlay/div/div/bx-login-overlay/bx-overlay-container/div/bx-overlay-body/section/bx-login-placeholder/bx-login/div/bx-form/form/bx-form-group/div/bx-input-field-container[1]/div/input")
    password_field = robust_wait_for_element(driver, By.XPATH, "/html/body/bx-site/bx-overlay-ch/bx-overlay/div/div/bx-login-overlay/bx-overlay-container/div/bx-overlay-body/section/bx-login-placeholder/bx-login/div/bx-form/form/bx-form-group/div/bx-input-field-container[2]/div/input")
    login_button = robust_wait_for_element(driver, By.XPATH, "/html/body/bx-site/bx-overlay-ch/bx-overlay/div/div/bx-login-overlay/bx-overlay-container/div/bx-overlay-body/section/bx-login-placeholder/bx-login/div/bx-form/form/div[2]/button")
    
    username_field.send_keys(username)
    password_field.send_keys(password)
    login_button.click()
    time.sleep(10)

# Initialize WebDriver
if use_debugging_mode:
    subprocess.Popen([r'C:/Program Files/Google/Chrome/Application/chrome.exe', '--remote-debugging-port=9222'])
    time.sleep(5)

chrome_options = Options()
if use_debugging_mode:
    chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

driver = webdriver.Chrome(options=chrome_options)

# Check if already logged in
if not skip_login:
    if not is_already_logged_in(driver):
        login_to_site(driver, username, password)


In [ ]:
# Initialize WebDriver
chrome_options = Options()
if use_debugging_mode:
    chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

driver = webdriver.Chrome(options=chrome_options)

# Check if already logged in
if not skip_login:
    if not is_already_logged_in(driver):
        login_to_site(driver, username, password)

# Navigate to the UFC/MMA betting page
driver.get("https://www.bovada.lv/sports/ufc-mma")
time.sleep(webpage_load_time)  # Wait for the UFC/MMA betting page to load

# Load the DataFrame from the uploaded CSV
csv_path = './Parlays/NewParlays.csv'
df = pd.read_csv(csv_path)

# Fetch and parse the page content
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    fighters = row['Parlay'].split(', ')
    bet_amount = round(row['Bet Size'], 2)
    bet_amount = max(bet_amount, 0.50)  # Ensure a minimum bet amount of 0.50
    
    # Initialize variables for verification attempts
    verification_attempts = 0
    verified = False
    
    while verification_attempts < 2 and not verified:
        # Locate the sections containing fights using BeautifulSoup
        fight_sections = soup.find_all("header", class_="event-title")
        
        for section in fight_sections:
            fighters_in_section = section.find_all("span", class_="name")
            fighter_names = [fighter.text.strip() for fighter in fighters_in_section]
            
            for idx, name in enumerate(fighter_names):
                best_match = max(fighters, key=lambda x: fuzz.partial_ratio(x, name))
                
                if fuzz.partial_ratio(best_match, name) > fuzzy_matching_pct:
                    bet_buttons = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.XPATH, f"//span[text()='{name}']/ancestor::header/following-sibling::sp-outcomes//button[@class='bet-btn']"))
                    )
                    
                    if idx < len(bet_buttons):
                        bet_button = bet_buttons[idx]
                        ActionChains(driver).move_to_element(bet_button).click().perform()
                        time.sleep(click_load_time)
                    else:
                        print(f"Expected button for {name} not found. Number of available buttons: {len(bet_buttons)}. Skipping this fighter.")
        
        # Verify the bet slip
        is_verified, mismatch_name = verify_bet_slip(driver, fighters)
        
        if is_verified:
            bet_input = robust_wait_for_element(driver, By.XPATH, "//input[@type='number']")
            bet_input.clear()
            bet_input.send_keys(str(bet_amount))
            time.sleep(click_load_time)
            
            # Place the bet if not in testing mode
            if not testing_mode:
                place_bet_button = robust_wait_for_element(driver, By.CSS_SELECTOR, ".place-bets.custom-cta.primary.cta-large.betTabs")
                ActionChains(driver).move_to_element(place_bet_button).click().perform()
                time.sleep(click_load_time)
                
                if is_bet_successful(driver):
                    print(f"Successfully placed bet of {bet_amount} for {', '.join(fighters)}.")
                    verified = True
                    
                    # Clear the board by clicking 'Continue Betting'
                    try:
                        continue_betting_button = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, "button.continue-betting.custom-cta.primary.cta-large"))
                        )
                        ActionChains(driver).move_to_element(continue_betting_button).click().perform()
                        time.sleep(click_load_time)
                    except TimeoutException:
                        print("Could not find 'Continue Betting' button. Proceeding anyway.")
                else:
                    print(f"Failed to place bet of {bet_amount} for {', '.join(fighters)}.")
        else:
            print(f"Verification failed. Mismatch or missing fighter: {mismatch_name}. Retrying...")
            verification_attempts += 1

    if not verified:
        print(f"Skipping bet due to failed verification after {verification_attempts} attempts.")
        continue  # Skip to the next iteration
    
    print(f"Attempted to place parlay bet of {bet_amount} for {', '.join(fighters)}. Moving on to next bet in 2 seconds.")
    time.sleep(2)
    
# Uncomment this line if you want to close the WebDriver after the loop
# driver.quit()
